In [13]:
using Turing
using StatsPlots
using Random

# サンプルデータの生成
Random.seed!(42)
# 真のパラメータ（3つの状態：表、裏、立つ）
true_p_head = 0.45    # 表が出る確率
true_p_tail = 0.35    # 裏が出る確率
# 残りは立つ確率 (0.20)

n_samples = 1000
# 多項分布からサンプリング
p = [true_p_head, true_p_tail, 1 - true_p_head - true_p_tail]
data = rand(Categorical(p), n_samples)

# モデルの定義
@model function coin_flip(data)
    # Dirichlet分布の事前分布（表、裏、立つ）
    p ~ Dirichlet(ones(3))

    # カテゴリカル分布での尤度
    for i in eachindex(data)
        data[i] ~ Categorical(p)
    end
end

# MCMCサンプリング
model = coin_flip(data)
chain = sample(model, NUTS(), 2000)

# 結果の可視化
plot(chain)

# 結果の要約
summary_df = describe(chain)
println("\n事後分布の要約:")
display(summary_df)

# 真の値との比較
println("\n真の値との比較:")
# 各パラメータの推定平均値を取得
mean_p_head = mean(chain[:p][:, 1])
mean_p_tail = mean(chain[:p][:, 2])
mean_p_stand = mean(chain[:p][:, 3])

println("True p_head: $(true_p_head), Estimated: $(mean_p_head)")
println("True p_tail: $(true_p_tail), Estimated: $(mean_p_tail)")
println("True p_stand: $(1-true_p_head-true_p_tail), Estimated: $(mean_p_stand)")

# 観測データと推定された確率の比較
outcomes = ["Head", "Tail", "Stand"]
observed_counts = counts(data, 1:3)
observed_probs = observed_counts ./ length(data)
estimated_probs = [mean_p_head, mean_p_tail, mean_p_stand]

bar(outcomes, observed_probs, alpha=0.5, label="Observed")
bar!(outcomes, estimated_probs, alpha=0.5, label="Estimated")
ylabel!("Probability")
title!("Observed vs Estimated Probabilities")


┌ Info: Found initial step size
└   ϵ = 0.2
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00



事後分布の要約:


2-element Vector{ChainDataFrame}:
 Summary Statistics (3 x 8)
 Quantiles (3 x 6)


真の値との比較:


LoadError: ArgumentError: index p not found